In [1]:
import pandas as pd
# Load datasets
orders = pd.read_csv(r"C:\Users\Ms Trinh\OneDrive - SRITHAI (VIET NAM) Co.Ltd\brazilian-ecommerce_2020\olist_orders_dataset.csv", parse_dates=['order_purchase_timestamp'])
order_items = pd.read_csv(r"C:\Users\Ms Trinh\OneDrive - SRITHAI (VIET NAM) Co.Ltd\brazilian-ecommerce_2020\olist_order_items_dataset.csv")
order_payments = pd.read_csv(r"C:\Users\Ms Trinh\OneDrive - SRITHAI (VIET NAM) Co.Ltd\brazilian-ecommerce_2020\olist_order_payments_dataset.csv")
customers = pd.read_csv(r"C:\Users\Ms Trinh\OneDrive - SRITHAI (VIET NAM) Co.Ltd\brazilian-ecommerce_2020\olist_customers_dataset.csv")


# EDA

## Explore orders

In [2]:
print({'no_rows': len(orders),
      'no_customer_id': orders['customer_id'].nunique()})

{'no_rows': 99441, 'no_customer_id': 99441}


In [ ]:
print({
    'status': orders['order_status'].unique(),
    'status_counts': orders['order_status'].value_counts(),
    'status_rates': orders['order_status'].value_counts(normalize=True).round(5)*100  
}
      )

{'status': array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object), 'status_counts': order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64, 'status_rates': order_status
delivered      97.020
shipped         1.113
canceled        0.629
unavailable     0.612
invoiced        0.316
processing      0.303
created         0.005
approved        0.002
Name: proportion, dtype: float64}


In [11]:
print(orders.isnull().sum())

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


- The data seems to show only one-time customer_id as the number of rows equals to the number of customer_id
- The overall successful delivery rate is more than 97 percents. We will set and retrieve the values ​​that have order_status as "delivered“
- 

In [13]:
# Clean orders
orders = orders[orders['order_status'].isin(['delivered'])]
orders = orders[['order_id', 'customer_id', 'order_purchase_timestamp']]

## Explore orders_payments

In [14]:
print({
    'payment_type': order_payments['payment_type'].unique(),
    'payment_type_counts': order_payments['payment_type'].value_counts(),
    'payment_type_rates': order_payments['payment_type'].value_counts(normalize=True).round(5)*100 
    })

{'payment_type': array(['credit_card', 'boleto', 'voucher', 'debit_card', 'not_defined'],
      dtype=object), 'payment_type_counts': payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64, 'payment_type_rates': payment_type
credit_card    73.922
boleto         19.044
voucher         5.559
debit_card      1.472
not_defined     0.003
Name: proportion, dtype: float64}


- Credit_card and boleto account for approximately 93% of the payment type
- We will set and retrieve the values ​​that not have payment_type as “not_defined“

In [15]:
# Clean order_payments
valid_types = ['credit_card', 'boleto', 'voucher', 'debit_card']
order_payments = order_payments[order_payments['payment_type'].isin(valid_types)]
payments_cleaned = order_payments.groupby('order_id')['payment_value'].sum().reset_index()

## Explore customers

In [16]:
summary = {
    'no_rows': len(customers),
    'no_customer_id': customers['customer_id'].nunique(),
    'no_unique_customer_id': customers['customer_unique_id'].nunique(),
    'no_distinct_city': customers['customer_city'].nunique(),
    'no_distinct_state': customers['customer_state'].nunique()
}
summary_df = pd.DataFrame([summary])
print(summary_df)

   no_rows  no_customer_id  no_unique_customer_id  no_distinct_city  \
0    99441           99441                  96096              4119   

   no_distinct_state  
0                 27  


no_customer_id = 99,441 > 96,096 = no_unique_customer_id so some customers might have more than one customer_id

In [17]:
# Clean customers
customers = customers.dropna(subset=['customer_state', 'customer_city'])
customers = customers[['customer_unique_id','customer_id', 'customer_state', 'customer_city']]

# RFM Analysis
To calculate recency, frequency and monetary value we will use unsupervised learning method to scoring customers.

### Merge datasets to create RFM DataFrame
RFM DataFrame is taken from 3 tables: orders, order_payments, customers. Therefore we will join these 2 tables together based on the key column customerID. 

In [18]:
df_rfm_raw = orders.merge(order_items, on='order_id', how='left') \
                   .merge(payments_cleaned, on='order_id', how='left') \
                   .merge(customers, on='customer_id', how='left')

### Define analysis date

In [19]:
analysis_date = df_rfm_raw['order_purchase_timestamp'].max()

### Recency
We calculate Recency by measure the time of the last order of minus the most recent purchase day.

In [20]:
recency_df = df_rfm_raw.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
recency_df['Recency'] = (analysis_date - recency_df['order_purchase_timestamp']).dt.days

### Frequency
We count the total time that customer buy our products.

In [21]:
frequency_df = df_rfm_raw.groupby('customer_unique_id')['order_id'].nunique().reset_index()
frequency_df.rename(columns={'order_id': 'Frequency'}, inplace=True)

### Monetary
We count total money that customers have spent on our products.

In [22]:
monetary_df = df_rfm_raw.groupby('customer_unique_id')['payment_value'].sum().reset_index()
monetary_df.rename(columns={'payment_value': 'Monetary'}, inplace=True)

In [23]:
rfm = recency_df[['customer_unique_id', 'Recency']] \
    .merge(frequency_df, on='customer_unique_id') \
    .merge(monetary_df, on='customer_unique_id')

### Score metrics by IQR
After calculate R, F, M values we score customers based on IQR, divide data set into 4 ranges with highest score is 4 and lowest is 1 we have these quartiles.

In [24]:
rfm['R_score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

In [ ]:
rfm['RFM_Score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

### Segment as BCG Matrix degrees
We will segment customers into 4 BCG groups based on their RFM value:
- STARS: 444, 443, 434, 344, 433, 343, 334

- QUESTION MARKS: 224, 214, , 223, 213, 323, 423, 413, 233, 243, 313, 234, 314, 244, 414, 333, 423, 324, 424

- CASH COWS
331, 341, 431, 441, 332, 342, 432, 442, 231,322, 242, 232, 422

- PET(DOGS): 112, 121, 211, 122, 221, 212, 222, 111, 241, 141, 131, 311, 321, 312, 132, 142, 123, 113, 114, 124, 143, 133,
134, 144, 411, 412, 421


In [26]:
def segment_customer(rfm_code):
    if rfm_code in ['444', '443', '434', '344', '433', '343', '334']:
        return 'STAR'
    elif rfm_code in ['224', '214', '223', '213', '323', '423', '413', '233', '243', 
                      '313', '234', '314', '414', '244', '333', '424', '324']:
        return 'QUESTION MARK'
    elif rfm_code in ['331', '341', '431', '441', '332', '342', '432', '442', 
                      '231', '322', '242', '232', '422']:
        return 'CASH COW'
    else:
        return 'DOGS'

In [27]:
rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)
rfm.to_excel("rfm_customer_segmentation.xlsx", index=False)